In [1]:
# Standard Libarary Imports
import os
import sys
import math
import random
sys.path.append("..")
# External Libaray Imports
import numpy as np
from tqdm import trange
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
# Deep Learning and Bayesian Deep Learning Imports
import deepbayes_prealpha
import deepbayes_prealpha.optimizers as optimizers

seed_value = 102
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
pos_ind = 0
EPOCHS = 50
HORIZON = 50
EXPLORE = 15
C_STATES = 35
EPISODES = 20 # 10 for all but three layers
SPEED_LIMIT = 0.5
GOAL_STATE = [-0.1,-0.1]
OBSTACLES = np.asarray([[0.4, 0.4]])
OBSTACLE = OBSTACLES[0]
ATTRACTORS = np.asarray([[0.1, 0.0], [0.0, 0.1]])

In [3]:
"""
The handcrafted 2D point environment we consider
"""
class P2D_env:
    def __init__(self):
        self.x = 0.80  + random.uniform(-0.1, 0)
        self.y = 0.80  + random.uniform(-0.1, 0)  #np.random.normal(-0.05, 0.025)
        self.dx = 0.0
        self.dy = 0.0
        # Point Mass Dynamics Control:
        self.h = 0.35      # time resolution
        self.T = HORIZON       # Time for each trajectory
        self.eta = 0.75    # friction coef
        self.m = 4.0      # mass of the car
        self.t = 0        # current time

    def step(self, act):
        if(self.t > self.T):
            sys.exit(-1)
        pt = np.asarray([self.x, self.y])
        vt = np.asarray([self.dx, self.dy])
        ut = np.asarray(act)
        vt_1 = ((1-((self.h*self.eta)/self.m))*vt) + ((self.h/self.m)*ut)
        pt_1 = pt + (self.h*vt_1)
        if(np.linalg.norm(pt_1 - OBSTACLE) < 0.05):
            print("COLLIDED")
            self.dx, self.dy = 0, 0
        self.x, self.y = pt_1[0], pt_1[1]
        self.dx, self.dy = vt_1[0], vt_1[1]
        self.t += 1
        state = np.asarray([self.x, self.y, self.dx, self.dy])
        return state, np.linalg.norm(pt) - np.linalg.norm(pt_1), self.complete(), 0

    def complete(self):
        state = np.asarray([self.x, self.y])
        if(self.x < 0.05 and self.y < 0.05):
            return True
        if(self.x < 0.0 or self.y < 0.0):
            return True
        #if(np.linalg.norm(state) < 0.05):
        #    return True
        if(self.t==self.T):
            return True
    def reset(self):
        self.x = 0.90  + random.uniform(-0.3, 0)
        self.y = 0.90  + random.uniform(-0.3, 0)
        self.dx = 0.0
        self.dy = 0.0
        self.t = 0
        return np.asarray([self.x, self.y, self.dx, self.dy])
    def action_space_sample(self):
        return np.random.rand(2)
    def observation_space_sample(self):
        return np.random.rand(4)

In [4]:
env = P2D_env()
state_0 = env.reset()
action_dims = len(env.action_space_sample())
observe_dims = len(env.observation_space_sample())
model_input_dims = observe_dims + action_dims
model_output_dims = observe_dims
control_input_dims = observe_dims
control_output_dims = action_dims

In [5]:
model = Sequential()
model.add(Dense(12, input_shape=(model_input_dims,), activation="sigmoid", dtype='float32'))
model.add(Dense(12, activation="sigmoid"))
model.add(Dense(12, activation="sigmoid"))
model.add(Dense(model_output_dims, activation="linear"))
learning_rate = 0.02; decay=0.0
#opt = optimizers.VariationalOnlineGuassNewton()
opt = optimizers.StochasticWeightAveragingGaussian()
mse_loss = tf.keras.losses.mean_squared_error
def cust_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    loss = tf.where(tf.math.reduce_all(y_pred > y_true, 1), 2.5*mse_loss(y_true, y_pred), mse_loss(y_true, y_pred))
    return loss
loss = cust_loss
#bayes_model = opt.compile(model, loss_fn=loss, epochs=35, learning_rate=learning_rate,
#                          decay=decay, inflate_prior=0.001, mode='regression', robust_train=10, linear_schedule=False)
bayes_model = opt.compile(model, loss_fn=loss, epochs=35, learning_rate=learning_rate,
                          decay=decay, inflate_prior=0.1, mode='regression', robust_train=0, linear_schedule=False)


opt.epsilon = 0.05

This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
deepbayes: Using implicit prior
(6, 12) 0.12909944487358055
(12, 12) 0.09128709291752768
(12, 12) 0.09128709291752768
(12, 4) 0.09128709291752768
deepbayes: Using implicit prior
(6, 12) 0.12909944487358055
(12, 12) 0.09128709291752768
(12, 12) 0.09128709291752768
(12, 4) 0.09128709291752768


2023-03-02 21:15:08.193178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def tf_reward_function(state_0, dyn_pred, goal):
    state_0 = tf.squeeze(state_0)
    dyn_pred = tf.squeeze(dyn_pred)
    state_0 = tf.cast(state_0, dtype=tf.float64)
    dyn_pred = tf.cast(dyn_pred, dtype=tf.float64)
    state_1 = state_0 + dyn_pred
    d1 = tf.norm(state_0[:,pos_ind:pos_ind+2] - goal, axis=1)
    d2 = tf.norm(state_1[:,pos_ind:pos_ind+2] - goal, axis=1)
    attractor_reward = 0
    for a in ATTRACTORS:
        #attractor_reward -= 0.5/(tf.norm(a - state_1[:,pos_ind:pos_ind+2], axis=1))
        dist_a1 = tf.norm(a - state_1[:,pos_ind:pos_ind+2], axis=1)
        non_reward = tf.cast(tf.ones(dist_a1.shape)*100, tf.float64)
        dist_a1 = tf.where(dist_a1 < 0.45, dist_a1, non_reward)
        value = 15/(dist_a1)
        attractor_reward -= value
    obstacle_penalty = 0
    for o in OBSTACLES:
        dist_o1 = tf.norm(o - state_0[:,pos_ind:pos_ind+2], axis=1)
        dist_o2 = tf.norm(o - state_1[:,pos_ind:pos_ind+2], axis=1)
        non_error = tf.cast(tf.ones(dist_o2.shape)*100, tf.float64)
        dist_o2 = tf.where(dist_o2 < 0.35, dist_o2, non_error)
        value = 12.5/(dist_o2)
        obstacle_penalty += value
    #return (-1.5*(d2-d1)) + (0.3/o2)
    return (-1.0/(d2-d1)) + obstacle_penalty + attractor_reward


In [7]:
ACTION_MATRIX = (np.random.rand(C_STATES, C_STATES, action_dims)*-0.5)

def return_AM_action(state_0, ACTION_MATRIX):
    #ind_i = math.floor(state_0[0]*C_STATES)
    #ind_j = math.floor(state_0[1]*C_STATES)
    ind_i = round(state_0[0]*C_STATES)
    ind_j = round(state_0[1]*C_STATES)
    a, b = random.uniform(-0.4, 0), random.uniform(-0.4, 0)
    retval = [a, b]
    try:
        retval =  ACTION_MATRIX[ind_i][ind_j]
    except:
        if(ind_i >= C_STATES*0.9 and ind_j >= C_STATES*0.9):
            retval = [-0, -0]
        elif(ind_j >= C_STATES*0.9):
            retval = ACTION_MATRIX[ind_i][C_STATES-1]
            retval[1] = -0
        elif(ind_j >= C_STATES*0.9):
            retval = ACTION_MATRIX[C_STATES-1][ind_j]
            retval[0] = -0
    return retval

In [8]:
# Define function to sample {(s,a) -> s} dataset
def gen_dynamics_dataset(env, size, dyn_model, X=[], y=[], fully_rand=False):
    X_train = list(X); y_train = list(y)
    appended = 0
    for traj in trange(size, desc="Generating Dynamics Dataset"):
        done = False
        prev_obs = env.reset()
        #prev_obs = np.concatenate((obs,env.robot_pos))
        while(not done):
            if(fully_rand==True):
                act = env.action_space_sample()
            else:
                #print("MAKING PREDICTION: ", fully_rand==False)
                act = return_AM_action(prev_obs, ACTION_MATRIX)
            obs, reward, done, collisions = env.step(act)
            #obs = np.concatenate((obs, env.robot_pos))
            x = np.concatenate((prev_obs,act))
            y = obs - prev_obs
            X_train.append(x); y_train.append(y)
            prev_obs = obs
    return np.asarray(X_train), np.asarray(y_train)

In [9]:
# Define a metric for how we are doing on the reward signal
def estimate_reward(env, dyn_model, size=5):
    reward = 0
    final_state = []
    rot_info = []
    saved_trajectories = []
    for traj in trange(size):
        done = False
        t = []
        prev_obs = env.reset()
        #prev_obs = env.robot_pos
        counter = 0
        while(not done):
            t.append(prev_obs)
            act = return_AM_action(prev_obs, ACTION_MATRIX)
            obs, r, done, collisions = env.step(act)
            #obs = env.robot_pos
            print("STATE: %s ACT: %s "%(obs, act))
            rot_info.append(np.squeeze(act)[1])
            reward += r; counter += 1
            prev_obs = obs
        final_state.append(obs)
        saved_trajectories.append(t)
        print("final state ", obs[pos_ind:pos_ind+2])
    reward/=size
    np.save("LearnedSystem/simulations", saved_trajectories)
    return reward, np.mean(final_state, axis=0), np.mean(rot_info)

In [10]:
def action_gradient(model, inp, loss_fn, w):
    gradient_sum = tf.zeros(np.asarray(inp).shape)
    inp = tf.convert_to_tensor(inp)
    model.model.set_weights(w)
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(inp)
        predictions = model.model(inp)
        loss = loss_fn([tf.squeeze(inp)[:,0:-2]], predictions, GOAL_STATE)
    inp_gradient = tape.gradient(loss, inp)
    inp_gradient = tf.cast(inp_gradient, 'float32')
    return tf.squeeze(inp_gradient)[:,-2:]

def update_AM(env, bayes_model, ACTION_MATRIX, iterations=25, learning_rate=0.01):
    print("UPDATING THE ACTION MATRIX")
    #if(loss is None):
    loss = tf_reward_function # This is gobally defined
    for _ in trange(iterations):
        X_data = []
        Y_data = []
        for i in range(len(ACTION_MATRIX)):
            for j in range(len(ACTION_MATRIX[0])):
                a, b = random.uniform(-0.4, 0), random.uniform(-0.4, 0)
                state_vars = [i/C_STATES, j/C_STATES, a, b]
                action_vars = ACTION_MATRIX[i][j]
                state = np.concatenate((state_vars, action_vars))
                X_data.append(state)
        grads = action_gradient(bayes_model, X_data, loss, bayes_model.sample())
        #grads = np.clip(grads, -0.75, 0.75)
        update = np.zeros((C_STATES, C_STATES, 2))
        for i in range(len(ACTION_MATRIX)):
            for j in range(len(ACTION_MATRIX[0])):
                update[i][j] = grads[i*C_STATES + j]
        ACTION_MATRIX -= learning_rate * np.asarray(update)
        for i in range(len(ACTION_MATRIX)):
            for j in range(len(ACTION_MATRIX[0])):
                if(np.linalg.norm(ACTION_MATRIX[i][j]) > SPEED_LIMIT):
                    ACTION_MATRIX[i][j] /= np.linalg.norm(ACTION_MATRIX[i][j])
                    ACTION_MATRIX[i][j] *= SPEED_LIMIT

In [11]:
X_dyn_train, y_dyn_train = gen_dynamics_dataset(env, 20, bayes_model, fully_rand=True)
bayes_model.train(X_dyn_train, y_dyn_train, X_dyn_train[0:100], y_dyn_train[0:100])
bayes_model.epochs = int(0.75*EPOCHS)
print("After Initialization REWARD: ", estimate_reward(env, bayes_model, 2))
for iteration in range(EPISODES):
    X_dyn_train, y_dyn_train = gen_dynamics_dataset(env, EXPLORE, bayes_model, X_dyn_train, y_dyn_train)
    bayes_model.train(X_dyn_train, y_dyn_train, X_dyn_train[0:100], y_dyn_train[0:100])
    reward = estimate_reward(env, bayes_model, 10)
    print("After %s REWARD: "%(iteration), reward)
    update_AM(env, bayes_model, ACTION_MATRIX)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 167.69it/s]


Epoch 1, loss: 0.329, err: 0.574, val_loss: 0.003, val_err: 0.050


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 201.83it/s]


Epoch 2, loss: 0.002, err: 0.048, val_loss: 0.002, val_err: 0.049


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 204.12it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 208.12it/s]


Epoch 39, loss: 0.000, err: 0.012, val_loss: 0.000, val_err: 0.019


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 585.62it/s]


STATE: [ 0.59535026  0.80084368 -0.02800898 -0.03360892] ACT: [-0.32010258 -0.38410199] 
STATE: [ 0.57638731  0.77808939 -0.05417986 -0.06501226] ACT: [-0.32010258 -0.38410199] 
STATE: [ 0.54614983  0.74801087 -0.08639281 -0.08593862] ACT: [-0.40878291 -0.28791758] 
STATE: [ 0.50388548  0.71372896 -0.12075528 -0.09794832] ACT: [-0.45750848 -0.2017077 ] 
STATE: [ 0.4491081   0.68080712 -0.15650681 -0.09406239] ACT: [-0.49915535 -0.02905059] 
STATE: [ 0.38370844  0.65573335 -0.18685616 -0.07163935] ACT: [-0.46422988  0.18571651] 
STATE: [ 0.30729574  0.63278736 -0.21832202 -0.06555997] ACT: [-0.4997519   0.01574914] 
STATE: [ 0.22059934  0.61200705 -0.24770399 -0.05937231] ACT: [-0.49953555  0.0215461 ] 
STATE: [ 0.12518133  0.58741416 -0.27262288 -0.07026541] ACT: [-0.47056522 -0.16902181] 
STATE: [ 0.03565121  0.54912724 -0.25580035 -0.1093912 ] ACT: [-0.01220967 -0.4998509 ] 
STATE: [-0.04578542  0.4982019  -0.23267607 -0.14550097] ACT: [ 0.07242714 -0.4947265 ] 
final state  [-0.0457

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.90it/s]


In [12]:
bayes_model.save("MinimalModel")

('batch_size', 64)
('learning_rate', 0.02)
('decay', 0.0)
('epochs', 37)
('inflate_prior', 0.1)
('input_noise', 0.0)
('robust_train', 0)
('epsilon', 0.05)
('robust_lambda', 0.5)
('loss_monte_carlo', 2)
('input_upper', inf)
('input_lower', -inf)
('record_epochs', 2)
('full_covar', 0)
('expl_lr', 0.0004)


/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [13]:
np.save("MinimalAM", ACTION_MATRIX)